## 5%数据集

In [1]:
import numpy as np
import pandas as pd
import jieba
import csv
import sklearn

## 载入数据

In [11]:
data = open('Data/5%/test.txt','r',encoding='utf-8')
f=open('Data/5%/5%.csv','a',newline='',encoding='utf-8')

csv_writer = csv.writer(f)

m=data.readlines()

csv_writer.writerow(["title","tag"])

for i in m:
    n=i.strip('\n')
    L=n.split('\t')
    # print(L)
    csv_writer.writerow(L)
    
f.close()
data.close()

In [12]:
data=pd.read_csv('Data/5%/5%.csv')
data.head()

,title,tag
0,今年一季度全国纪检监察机关处分省部级干部16人,0
1,习近平：携手迎接挑战 合作开创未来,0
2,瞭望丨引领人类海洋文明发展方向,0
3,广东省老区湾区同携手 接续奋进新时代,0
4,关注全民健身：健身场地少、消费高等难题怎么破？,0


## 划分训练集测试集

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data['title'], data['tag'], random_state=1)



In [14]:
print(x_test.shape[0])
print(type(y_test))

180
<class 'pandas.core.series.Series'>


## 分词

In [15]:
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df

x_train_fenci = fenci(x_train)
x_train_fenci[:5]


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\32852\AppData\Local\Temp\jieba.cache
Loading model cost 0.697 seconds.
Prefix dict has been built successfully.


262                 妖股 第二次 砸 盘 ， 欢乐 海岸 还会来 反包 吗 ？
388    国家 奥运 精英 运动员 代表团 将 于 12 月 19 日至 21 日 访问 澳门
352              美 发布 所谓 涉港 报告 妄评 我治港 政策   外交部 回应
366          中方 ： 奉劝 英国 有关 政客 不要 在 台湾 问题 上 “ 玩火 ”
259                 商标 转让 和 商标 变更 一样 吗 ？ 不 一样 吗 ？
Name: title, dtype: object

## 停词

In [16]:
infile = open("hit_stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

## 向量化

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stopwords, max_features=5000)
vectorizer.fit(x_train_fenci)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CountVectorizer(max_features=5000,
                stop_words=['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→',
                            '℃', '&', '*', '一一', '~~~~', '’', '.', '『', '.一',
                            './', '--', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］',
                            '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', ...])

In [18]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
#模型训练
classifier.fit(vectorizer.transform(x_train_fenci), y_train)
#使用训练好的模型进行预测
classifier.score(vectorizer.transform(fenci(x_test)), y_test)

0.5833333333333334

## 预测

In [19]:
from sklearn.metrics import classification_report
predict=classifier.predict(vectorizer.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.43      0.80      0.56        15
           1       0.70      0.59      0.64        27
           2       0.86      0.33      0.48        36
           3       0.50      0.37      0.42        19
           4       0.72      0.54      0.62        24
           5       0.57      0.92      0.71        13
           6       0.45      0.82      0.58        11
           7       0.67      0.64      0.65        22
           8       0.48      0.77      0.59        13

    accuracy                           0.58       180
   macro avg       0.60      0.64      0.58       180
weighted avg       0.65      0.58      0.57       180



In [20]:
f=open('test_5.txt','r',encoding='utf-8')

L=[]
m=f.readlines()
for i in m:
    n=i.strip('\n')
    x=n.split('\t')
    L.append(x[1])

data_to_predict=pd.Series(L)
# data_to_predict=np.matrix(data_to_predict)

In [21]:
print(data_to_predict.shape)

(603,)


In [22]:
predict=classifier.predict(vectorizer.transform(fenci(data_to_predict)))

In [23]:
print(predict)

[5 7 4 6 3 1 2 1 1 2 7 4 4 7 3 4 1 2 1 5 3 4 2 7 4 3 2 1 3 4 4 4 1 8 0 5 2
 2 2 3 5 7 6 6 5 7 4 3 3 1 5 5 5 7 3 5 6 5 2 4 3 0 1 1 2 0 3 7 2 0 6 0 1 0
 2 8 2 5 5 3 4 2 6 2 4 3 2 3 3 6 2 3 3 2 4 0 3 2 3 5 5 5 7 4 4 3 1 2 7 4 2
 0 2 3 3 5 4 8 8 7 0 1 5 0 0 2 2 4 3 1 1 4 4 4 1 6 7 4 0 2 7 3 2 4 5 2 4 1
 0 3 4 3 5 3 0 1 8 7 8 1 2 6 3 5 2 3 1 2 3 1 6 7 6 3 1 2 0 3 2 2 7 6 4 3 1
 2 2 6 5 2 5 8 2 3 2 2 2 4 1 1 7 2 1 4 1 4 1 5 2 2 4 0 2 6 4 2 4 4 3 8 1 2
 5 3 3 2 2 4 3 2 5 4 3 2 0 7 6 2 3 2 4 3 2 2 4 4 5 4 7 5 4 6 2 3 7 6 5 3 2
 2 3 8 4 4 1 2 3 7 0 2 4 6 1 3 4 2 4 6 7 7 2 1 4 2 0 7 6 3 2 4 0 4 6 2 0 0
 2 8 5 1 6 8 3 2 8 2 0 8 8 0 7 1 2 5 4 2 1 0 6 6 6 8 4 3 3 2 5 0 5 6 4 4 2
 8 3 3 0 7 0 0 2 2 2 2 6 1 4 7 6 2 4 3 1 4 6 4 4 4 1 2 0 1 1 8 0 3 2 1 5 1
 4 0 6 5 1 2 5 1 6 5 2 4 3 5 2 0 0 4 6 4 4 0 1 1 4 4 6 4 6 2 3 3 4 2 1 3 5
 0 4 3 5 0 5 2 5 4 5 4 4 3 7 0 7 1 0 7 4 3 3 1 5 6 8 5 3 2 8 5 6 4 2 8 2 6
 2 2 2 0 4 7 0 5 4 3 4 5 3 2 0 1 0 0 6 2 4 0 4 4 7 2 1 0 0 3 1 6 4 1 8 7 2
 2 1 0 5 1 5 2 0 5 1 2 0 

## TF-IDF预处理

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
#使用tf-idf把文本转为向量
tv = TfidfVectorizer(stop_words=stopwords, max_features=5000, lowercase = False)
tv.fit(x_train_fenci)
#模型训练
classifier.fit(tv.transform(fenci(x_train)), y_train)
#利用训练好的模型测试
classifier.score(tv.transform(fenci(x_test)), y_test)


D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.6055555555555555

In [25]:
from sklearn.metrics import classification_report
predict=classifier.predict(tv.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.43      0.80      0.56        15
           1       0.65      0.75      0.70        20
           2       0.86      0.32      0.46        38
           3       0.64      0.38      0.47        24
           4       0.72      0.54      0.62        24
           5       0.52      0.92      0.67        12
           6       0.50      0.91      0.65        11
           7       0.76      0.73      0.74        22
           8       0.52      0.79      0.63        14

    accuracy                           0.61       180
   macro avg       0.62      0.68      0.61       180
weighted avg       0.67      0.61      0.59       180



## N-gram 模型

In [26]:
tv_2gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,2),lowercase = False)
tv_2gram.fit(x_train_fenci)
#训练模型
clf_2gram = MultinomialNB()
clf_2gram.fit(tv_2gram.transform(fenci(x_train)), y_train)
#预测
clf_2gram.score(tv_2gram.transform(fenci(x_test)), y_test)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.5666666666666667

In [27]:
from sklearn.metrics import classification_report
predict=clf_2gram.predict(tv_2gram.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.39      0.85      0.54        13
           1       0.65      0.71      0.68        21
           2       0.86      0.28      0.42        43
           3       0.50      0.32      0.39        22
           4       0.72      0.57      0.63        23
           5       0.52      0.79      0.63        14
           6       0.45      0.90      0.60        10
           7       0.67      0.67      0.67        21
           8       0.48      0.77      0.59        13

    accuracy                           0.57       180
   macro avg       0.58      0.65      0.57       180
weighted avg       0.64      0.57      0.55       180



## 3-gram

In [28]:
tv_3gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,3),lowercase = False)
tv_3gram.fit(x_train_fenci)
#训练模型
clf_3gram = MultinomialNB()
clf_3gram.fit(tv_3gram.transform(fenci(x_train)), y_train)
#预测
clf_3gram.score(tv_3gram.transform(fenci(x_test)), y_test)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.5333333333333333

In [26]:
from sklearn.metrics import classification_report
predict=clf_3gram.predict(tv_3gram.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.64      0.66      0.65        56
           1       0.80      0.65      0.72        80
           2       0.73      0.79      0.76        68
           3       0.67      0.65      0.66        65
           4       0.76      0.80      0.78        70
           5       0.74      0.82      0.78        68
           6       0.54      0.75      0.63        53
           7       0.76      0.41      0.54        99
           8       0.57      0.86      0.69        42

    accuracy                           0.69       601
   macro avg       0.69      0.71      0.69       601
weighted avg       0.70      0.69      0.68       601

